#download data set

In [47]:
!pip install transformers

In [48]:
import pandas as pd
import numpy as np

In [49]:
setswana_url = 'https://raw.githubusercontent.com/NLPforLRLsProjects/SAfriSenti-Corpus/refs/heads/main/setswana_tweets.csv'
sesotho_url = 'https://raw.githubusercontent.com/NLPforLRLsProjects/SAfriSenti-Corpus/refs/heads/main/sesotho_tweets.csv'

#setswana_df = pd.read_csv(setswana_url)
sesotho_df = pd.read_csv(sesotho_url)

In [50]:
#setswana_df.shape

In [51]:
#setswana_df.head()

In [52]:
#setswana_df.shape

In [53]:
sesotho_df.head()

,sentence,Final_labels,predict_name
0,@user gwa tshwana rena ba bang a re kreye selo...,negative,Sesotho
1,e bata goal spurs,neutral,Sesotho
2,@user @user ke nahana taba eno ea ho batla ho ...,negative,multi
3,@user lotho hle empa fela ke ipotela,positive,Sesotho
4,@user @user keu utloa hantle,positive,Sesotho


#tokenize dataset

In [54]:
#setswana_df['Final_Label'].unique()

In [55]:
sesotho_df.head()

,sentence,Final_labels,predict_name
0,@user gwa tshwana rena ba bang a re kreye selo...,negative,Sesotho
1,e bata goal spurs,neutral,Sesotho
2,@user @user ke nahana taba eno ea ho batla ho ...,negative,multi
3,@user lotho hle empa fela ke ipotela,positive,Sesotho
4,@user @user keu utloa hantle,positive,Sesotho


In [56]:
sesotho_df.rename(columns={'Final_labels': 'Final_Label'}, inplace=True)

In [57]:
def label_to_int(x):
  if x == 'positive':
    return 1
  elif x == 'negative':
    return 0
  else:
    return 2

In [58]:
label_map = pd.DataFrame({
    'label': ['negative', 'positive', 'neutral'],
    'key': [0, 1, 2]
})

In [59]:
# value_counts = setswana_df['Final_Label'].value_counts().reset_index()
# value_counts.columns = ['label', 'count']
# value_counts_keys = pd.merge(label_map, value_counts, on='label', how='left').fillna(0)
# value_counts_keys['count'] = value_counts_keys['count'].astype(int)

In [60]:
value_counts_keys

,label,key,count
0,negative,0,1464
1,positive,1,953
2,neutral,2,583


In [61]:
value_counts = sesotho_df['Final_Label'].value_counts().reset_index()
value_counts.columns = ['label', 'count']
value_counts_keys = pd.merge(label_map, value_counts, on='label', how='left').fillna(0)
value_counts_keys['count'] = value_counts_keys['count'].astype(int)

In [62]:
value_counts_keys

,label,key,count
0,negative,0,1464
1,positive,1,953
2,neutral,2,583


In [63]:
#setswana_df['Final_Label'] = [label_to_int(x) for x in setswana_df['Final_Label']]
sesotho_df['Final_Label'] = [label_to_int(x) for x in sesotho_df['Final_Label']]

#tokenize using subword tokenization

In [64]:
from transformers import XLMRobertaTokenizer, AutoTokenizer

In [65]:
bpe_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
word_piece_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [66]:
#setswana_df['tokens_bpe'] = setswana_df['sentence'].apply(lambda x: tokenizer.tokenize(x))
sesotho_df['bpe_tokens'] = sesotho_df['sentence'].apply(lambda x: bpe_tokenizer.tokenize(x))
sesotho_df['word_piece_tokens'] = sesotho_df['sentence'].apply(lambda x: word_piece_tokenizer.tokenize(x))

In [67]:
#setswana_df.head()

In [68]:
sesotho_df.head()

,sentence,Final_Label,predict_name,bpe_tokens,word_piece_tokens
0,@user gwa tshwana rena ba bang a re kreye selo...,0,Sesotho,"[▁@, user, ▁gwa, ▁t, shwa, na, ▁rena, ▁ba, ▁ba...","[@, user, g, ##wa, ts, ##hwa, ##na, ren, ##a, ..."
1,e bata goal spurs,2,Sesotho,"[▁e, ▁bata, ▁goal, ▁, spur, s]","[e, bat, ##a, goal, spurs]"
2,@user @user ke nahana taba eno ea ho batla ho ...,0,multi,"[▁@, user, ▁@, user, ▁ke, ▁na, hana, ▁taba, ▁e...","[@, user, @, user, ke, nah, ##ana, tab, ##a, e..."
3,@user lotho hle empa fela ke ipotela,1,Sesotho,"[▁@, user, ▁lot, ho, ▁, hle, ▁emp, a, ▁fel, a,...","[@, user, lot, ##ho, h, ##le, em, ##pa, fe, ##..."
4,@user @user keu utloa hantle,1,Sesotho,"[▁@, user, ▁@, user, ▁ke, u, ▁ut, lo, a, ▁han,...","[@, user, @, user, ke, ##u, ut, ##lo, ##a, han..."


#process tokens

In [69]:
#setswana_encoding = tokenizer(setswana_df['sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)
sesotho_bpe_encoding = bpe_tokenizer(sesotho_df['sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)
sesotho_word_piece_encoding = word_piece_tokenizer(sesotho_df['sentence'].tolist(), padding=True, truncation=True, return_tensors='pt',max_length=64)

In [70]:
sesotho_bpe_encoding

{'input_ids': tensor([[    0,  1374, 65918,  ...,     1,     1,     1],
        [    0,    28,  8336,  ...,     1,     1,     1],
        [    0,  1374, 65918,  ..., 47130,  1777,     2],
        ...,
        [    0,   764,  8110,  ...,     1,     1,     1],
        [    0,  1374, 65918,  ...,     1,     1,     1],
        [    0,  1374, 65918,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [71]:
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import torch
from sklearn.model_selection import train_test_split



In [72]:
def train_val_dataloader(encodings,data_labels):
  input_ids = encodings
  labels = data_labels

  dataset = TensorDataset(input_ids, labels)
  train_size = int(0.8 * len(dataset))
  val_size = len(dataset) - train_size

  train_ds, val_ds = random_split(dataset, [train_size, val_size])

  train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
  val_dl = DataLoader(val_ds, batch_size=32)

  return train_dl, val_dl

#Create neural model

In [73]:
import torch.nn as nn
import torch.nn.functional as F

class TweetClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(TweetClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.fc1 = nn.Linear(embed_dim * 64, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)              # (batch_size, seq_len, embed_dim)
        flat = embedded.view(x.size(0), -1)       # Flatten: (batch_size, seq_len * embed_dim)
        out = F.relu(self.fc1(flat))              # (batch_size, hidden_dim)
        return self.fc2(out)                      # (batch_size, output_dim)


In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [75]:
def train_eval_model(model, epochs,train_dl,val_dl):
    for epoch in range(epochs):
      model.train()
      total_loss = 0

      for xb, yb, in train_dl:
          preds = model(xb)
          loss = loss_fn(preds, yb)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    model.eval()
    correct, total = 0, 0

    predictions = []
    true_labels = []

    with torch.no_grad():
        for xb, yb in val_dl:
            preds = model(xb)
            predicted = torch.argmax(preds, dim=1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)

            predictions.extend(predicted)
            true_labels.extend(yb)


    print(f"Validation Accuracy: {correct / total:.2%}")

    return predictions, true_labels


In [76]:
def metrics(predictions, true_labels):

  accuracy = accuracy_score(true_labels, predictions)
  micro_precision = precision_score(true_labels, predictions, average='micro')
  micro_recall = recall_score(true_labels, predictions, average='micro')
  micro_f1 = f1_score(true_labels, predictions, average='micro')

  print(f"Accuracy: {accuracy:.4%}")
  print()
  print(f"Micro-average Precision: {micro_precision:.4%}")
  print(f"Micro-average Recall: {micro_recall:.4%}")
  print(f"Micro-average F1-score: {micro_f1:.4%}")
  print()

  macro_precision = precision_score(true_labels, predictions, average='macro', zero_division=0)
  macro_recall = recall_score(true_labels, predictions, average='macro', zero_division=0)
  macro_f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)

  print(f"Macro-average Precision: {macro_precision:.4%}")
  print(f"Macro-average Recall: {macro_recall:.4%}")
  print(f"Macro-average F1-score: {macro_f1:.4%}")
  print()

  macro_precision = precision_score(true_labels, predictions, average='macro', zero_division=0)
  macro_recall = recall_score(true_labels, predictions, average='macro', zero_division=0)
  macro_f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)

  print(f"Macro-average Precision: {macro_precision:.4%}")
  print(f"Macro-average Recall: {macro_recall:.4%}")
  print(f"Macro-average F1-score: {macro_f1:.4%}")

## BPE model

In [77]:
model = TweetClassifier(vocab_size=bpe_tokenizer.vocab_size, embed_dim=64, hidden_dim=128, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [78]:
train_dl, val_dl = train_val_dataloader(sesotho_bpe_encoding['input_ids'], torch.tensor(sesotho_df['Final_Label'].tolist()))

### evaluate model

In [79]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 77.1394
Epoch 2, Loss: 64.1283
Epoch 3, Loss: 56.4273
Epoch 4, Loss: 47.1877
Epoch 5, Loss: 38.5509
Epoch 6, Loss: 31.1935
Epoch 7, Loss: 23.7428
Epoch 8, Loss: 20.9726
Epoch 9, Loss: 15.5457
Epoch 10, Loss: 12.6629
Validation Accuracy: 46.83%


In [80]:
metrics(np.array(predictions),np.array(true_labels))

Accuracy: 46.8333%

Micro-average Precision: 46.8333%
Micro-average Recall: 46.8333%
Micro-average F1-score: 46.8333%

Macro-average Precision: 41.0705%
Macro-average Recall: 40.7151%
Macro-average F1-score: 40.8324%

Macro-average Precision: 41.0705%
Macro-average Recall: 40.7151%
Macro-average F1-score: 40.8324%


# STF-idf

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

In [123]:
class STFIDFClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(STFIDFClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        out = F.relu(self.fc1(x))            # (batch_size, hidden_dim)
        return self.fc2(out)                      # (batch_size, output_dim)

In [124]:
sesotho_df

,sentence,Final_Label,predict_name,bpe_tokens,word_piece_tokens
0,@user gwa tshwana rena ba bang a re kreye selo...,0,Sesotho,"[▁@, user, ▁gwa, ▁t, shwa, na, ▁rena, ▁ba, ▁ba...","[@, user, g, ##wa, ts, ##hwa, ##na, ren, ##a, ..."
1,e bata goal spurs,2,Sesotho,"[▁e, ▁bata, ▁goal, ▁, spur, s]","[e, bat, ##a, goal, spurs]"
2,@user @user ke nahana taba eno ea ho batla ho ...,0,multi,"[▁@, user, ▁@, user, ▁ke, ▁na, hana, ▁taba, ▁e...","[@, user, @, user, ke, nah, ##ana, tab, ##a, e..."
3,@user lotho hle empa fela ke ipotela,1,Sesotho,"[▁@, user, ▁lot, ho, ▁, hle, ▁emp, a, ▁fel, a,...","[@, user, lot, ##ho, h, ##le, em, ##pa, fe, ##..."
4,@user @user keu utloa hantle,1,Sesotho,"[▁@, user, ▁@, user, ▁ke, u, ▁ut, lo, a, ▁han,...","[@, user, @, user, ke, ##u, ut, ##lo, ##a, han..."
...,...,...,...,...,...
2995,@user @user ntho diqala jwalo rumoursthe knes ...,0,Sesotho,"[▁@, user, ▁@, user, ▁n, tho, ▁di, qala, ▁j, w...","[@, user, @, user, nt, ##ho, di, ##qa, ##la, j..."
2996,@user iΓö£┬╗Γö¼Γò£m on leave ntse ke sheba hen...,0,Sesotho,"[▁@, user, ▁i, Γ, ö, £, ┬, ╗, Γ, ö, 1⁄4, Γ, ò,...","[@, user, [UNK], on, leave, nt, ##se, ke, she,..."
2997,he must come to botswana a re thuse that bogus...,0,multi,"[▁he, ▁must, ▁come, ▁to, ▁bot, s, wana, ▁a, ▁r...","[he, must, come, to, botswana, a, re, thus, ##..."
2998,@user wa bona ea hlokomela something fishy moh...,0,Sesotho,"[▁@, user, ▁wa, ▁bona, ▁ea, ▁h, loko, mela, ▁s...","[@, user, wa, bon, ##a, ea, h, ##lok, ##ome, #..."


## N-gram

In [125]:

# Using character n-grams for TfidfVectorizer directly
stfidf_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4), min_df=2) # Example char 2-4-grams, min_df to filter rare ones
stf_idf_matrix = stfidf_vectorizer.fit_transform(sesotho_df['sentence'])
stf_idf_vocab_size = len(stfidf_vectorizer.get_feature_names_out())

# Convert to PyTorch tensor
stf_idf_tensor = torch.tensor(stf_idf_matrix.toarray(), dtype=torch.float32)
print(stf_idf_tensor.shape)
print(stf_idf_vocab_size)

torch.Size([3000, 20380])
20380


In [126]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [127]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=64, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [128]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 75.5258
Epoch 2, Loss: 63.3999
Epoch 3, Loss: 47.3804
Epoch 4, Loss: 31.9987
Epoch 5, Loss: 20.9797
Epoch 6, Loss: 13.7715
Epoch 7, Loss: 9.3605
Epoch 8, Loss: 6.6437
Epoch 9, Loss: 4.8691
Epoch 10, Loss: 3.7451
Validation Accuracy: 53.83%


In [129]:
metrics(np.array(predictions),np.array(true_labels))

Accuracy: 53.8333%

Micro-average Precision: 53.8333%
Micro-average Recall: 53.8333%
Micro-average F1-score: 53.8333%

Macro-average Precision: 50.1039%
Macro-average Recall: 48.1809%
Macro-average F1-score: 48.3359%

Macro-average Precision: 50.1039%
Macro-average Recall: 48.1809%
Macro-average F1-score: 48.3359%


## BPE

In [130]:
subword_bpe_tokenised_documnets = [" ".join(s) for s in sesotho_df['bpe_tokens']]
subword_bpe_tokenised_documnets[0:3]

['▁@ user ▁gwa ▁t shwa na ▁rena ▁ba ▁bang ▁a ▁re ▁kre ye ▁se lo ▁mos',
 '▁e ▁bata ▁goal ▁ spur s',
 '▁@ user ▁@ user ▁ke ▁na hana ▁taba ▁en o ▁ea ▁ho ▁bat la ▁ho ▁khe tha ▁ho bane ▁re ▁she bile ▁our ▁own ▁benefits ▁re ▁le ▁bat ho ▁ke ▁e ona ▁e ▁sent seng ▁polo tik i ▁ea ▁les ot ho ▁so ▁we ▁went ▁behind ▁as sho les ▁for ▁years ▁co z ▁re ▁she bile ▁mele mo ▁e ▁direct ▁rele ▁bat ho ▁a ▁ee a ▁sebe let sa ▁n']

In [131]:

# Using character n-grams for TfidfVectorizer directly
stfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2)
stf_idf_matrix = stfidf_vectorizer.fit_transform(subword_bpe_tokenised_documnets)
stf_idf_vocab_size = len(stfidf_vectorizer.get_feature_names_out())


# Convert to PyTorch tensor
stf_idf_tensor = torch.tensor(stf_idf_matrix.toarray(), dtype=torch.float32)
print(stf_idf_tensor.shape)
print(stf_idf_vocab_size)


torch.Size([3000, 8887])
8887


In [132]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [133]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=64, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [134]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 78.3452
Epoch 2, Loss: 68.5257
Epoch 3, Loss: 53.6911
Epoch 4, Loss: 37.3525
Epoch 5, Loss: 24.8792
Epoch 6, Loss: 16.7893
Epoch 7, Loss: 11.5945
Epoch 8, Loss: 8.3698
Epoch 9, Loss: 6.3120
Epoch 10, Loss: 4.9122
Validation Accuracy: 55.33%


In [135]:
metrics(np.array(predictions),np.array(true_labels))

Accuracy: 55.3333%

Micro-average Precision: 55.3333%
Micro-average Recall: 55.3333%
Micro-average F1-score: 55.3333%

Macro-average Precision: 53.3762%
Macro-average Recall: 49.8557%
Macro-average F1-score: 50.8773%

Macro-average Precision: 53.3762%
Macro-average Recall: 49.8557%
Macro-average F1-score: 50.8773%


## Word piece

In [136]:
subword_word_piece_tokenised_documnets = [" ".join(s) for s in sesotho_df['word_piece_tokens']]
subword_word_piece_tokenised_documnets[0:3]

['@ user g ##wa ts ##hwa ##na ren ##a ba bang a re k ##rey ##e se ##lo mo ##s',
 'e bat ##a goal spurs',
 '@ user @ user ke nah ##ana tab ##a en ##o ea ho bat ##la ho k ##het ##ha ho ##bane re she ##bil ##e our own benefits re le bath ##o ke e ##ona e sent ##sen ##g polo ##ti ##ki ea les ##otho so we went behind asshole ##s for years co ##z re she ##bil ##e mel ##em ##o e direct re ##le bath ##o a ee ##a se ##bel ##ets ##a n']

In [137]:
train_dl, val_dl = train_val_dataloader(stf_idf_tensor, torch.tensor(sesotho_df['Final_Label'].tolist()))

In [138]:
model = STFIDFClassifier(input_dim=stf_idf_vocab_size, hidden_dim=64, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [139]:
predictions, true_labels = train_eval_model(model,10,train_dl,val_dl)

Epoch 1, Loss: 78.4820
Epoch 2, Loss: 68.4439
Epoch 3, Loss: 54.1073
Epoch 4, Loss: 38.5225
Epoch 5, Loss: 25.8637
Epoch 6, Loss: 17.3783
Epoch 7, Loss: 12.0046
Epoch 8, Loss: 8.6029
Epoch 9, Loss: 6.3949
Epoch 10, Loss: 4.9802
Validation Accuracy: 55.00%


In [140]:
metrics(np.array(predictions),np.array(true_labels))

Accuracy: 55.0000%

Micro-average Precision: 55.0000%
Micro-average Recall: 55.0000%
Micro-average F1-score: 55.0000%

Macro-average Precision: 54.7405%
Macro-average Recall: 48.0822%
Macro-average F1-score: 48.8849%

Macro-average Precision: 54.7405%
Macro-average Recall: 48.0822%
Macro-average F1-score: 48.8849%


# Word tokenization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')



tokenized = [word_tokenize(s.lower()) for s in sesotho_df['sentence'].tolist()]

# Build a vocabulary
vocab = {"<pad>": 0, "<unk>": 1}
for sent in tokenized:
    for word in sent:
        if word not in vocab:
            vocab[word] = len(vocab)


In [ ]:
SEQ_LEN = 64

def encode(sent):
    tokens = word_tokenize(sent.lower())
    token_ids = [vocab.get(tok, vocab["<unk>"]) for tok in tokens]
    if len(token_ids) < SEQ_LEN:
        token_ids += [vocab["<pad>"]] * (SEQ_LEN - len(token_ids))
    else:
        token_ids = token_ids[:SEQ_LEN]
    return token_ids



In [ ]:
class TweetDataset():
    def __init__(self, texts, labels):
        self.texts = texts
        self.encoded_texts = torch.tensor([encode(t) for t in texts])
        self.labels = labels

In [ ]:
dataset = TweetDataset(setswana_df['sentence'].tolist(), setswana_df['Final_Label'].tolist())

In [ ]:
dataset.texts[:5]

In [ ]:
dataset.encoded_texts[:5]

In [ ]:
model = TweetClassifier(vocab_size=len(vocab), embed_dim=64, hidden_dim=128, output_dim=3)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
train_dl, val_dl = train_val_dataloader(dataset.encoded_texts, torch.tensor(setswana_df['Final_Label'].tolist()))

In [ ]:
train_eval_model(model,10,train_dl,val_dl)